In [1]:
import os
import glob
import pandas as pd
import numpy as np
from multiprocessing import Pool

In [6]:
m6a_dir = "/home/galaxy/project/QTL_analysis/data/m6a_peak"
#
result_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/sQTL/sQTLSeekeR_Transcript-Ratio/m6a_qtl_txt"
qtl_dir = "/home/galaxy/data/QTL/sQTL/sQTLSeekeR_Transcript-Ratio/GRCh38_bed"
qtl_list = glob.glob("%s/*.bed" % qtl_dir)
postfix = ".bed"

In [3]:
def generate_map_dict():
    map_dict = {"Heart": "Heart", "Lung": "Lung", "Muscle": "Muscle"}
    return map_dict

def perform_intersection(eQTL, m6a):
    pre_eQTL, pre_m6a = os.path.basename(eQTL).split(postfix)[0], os.path.basename(m6a).split(".bed")[0]
    result_txt = os.path.join(result_dir, "%s-%s" % (pre_eQTL, os.path.basename(m6a).replace(".bed", ".txt")))
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (eQTL, m6a, result_txt))

In [7]:
map_dict = generate_map_dict()
for qtl in qtl_list:
    prefix = os.path.basename(qtl).split(".")[0].split("_")[0]
    if prefix in map_dict:
        m6a = "%s/%s.bed" % (m6a_dir, map_dict[prefix])
        print(os.path.basename(qtl), "\t", os.path.basename(m6a))
        perform_intersection(qtl, m6a)

Heart_Left_Ventricle.bed 	 Heart.bed
Muscle_Skeletal.bed 	 Muscle.bed
Lung.bed 	 Lung.bed


In [2]:
#########################################################################
data_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/sQTL/sQTLSeekeR_Transcript-Ratio/m6a_qtl_txt"
result_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/sQTL/sQTLSeekeR_Transcript-Ratio/m6a_qtl_bed"
qtl_list = glob.glob("%s/*.txt" % data_dir)
for qtl in qtl_list:
    tissue = os.path.basename(qtl).split("-")[-1].split(".")[0]
    df = pd.read_table(qtl, sep="\t", header=None)
    df["tissue"] = tissue
    df["info"]=df.iloc[:, 3]+";"+df.iloc[:, 4]+":"+df.iloc[:, 5].astype(str)+"-"+df.iloc[:, 6].astype(str)+";"+df.iloc[:, 7]
    result_bed = os.path.join(result_dir, os.path.basename(qtl).replace(".txt", ".bed"))
    df_bed = df.iloc[:, [0,1,2,-1]]
    df_bed.to_csv(result_bed, sep="\t", header=False, index=False)